In [8]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.quantization

In [9]:
# Define the ANN class
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to register hooks and capture data
def get_intermediate_outputs(model, input_data):
    intermediate_outputs = {}

    def hook_fn(module, input, output):
        class_name = module.__class__.__name__
        module_idx = len(intermediate_outputs)
        intermediate_outputs[f"{class_name}_{module_idx}"] = {
            "input": input,
            "output": output
        }

    hooks = []
    for layer in model.children():
        hook = layer.register_forward_hook(hook_fn)
        hooks.append(hook)

    # Forward pass
    with torch.no_grad():
        model(input_data)

    # Remove hooks
    for hook in hooks:
        hook.remove()

    return intermediate_outputs

In [10]:
# Instantiate the model
model = ANN()

# Prepare the model for static quantization
torch.backends.quantized.engine = 'qnnpack'
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model, inplace=True)

# Calibrate the model with representative data
input_tensor = torch.randn(100, 2)  # Use a larger batch for calibration
model(input_tensor)

# Convert the model to a quantized version
torch.quantization.convert(model, inplace=True)

# Define an input tensor with appropriate shape
input_tensor = torch.randint(0, 10, (3, 2), dtype=torch.float)

# Capture intermediate outputs
intermediate_outputs = get_intermediate_outputs(model, input_tensor)

# Save all tensors to files
d = "trace/"

torch.save(input_tensor, d + 'input_tensor.pt')
for key, data in intermediate_outputs.items():
    torch.save(data['input'], d + f"{key}_input.pt")
    torch.save(data['output'], d + f"{key}_output.pt")

print("Tensors have been saved.")

NotImplementedError: Could not run 'quantized::linear' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear' is only available for these backends: [MPS, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

MPS: registered at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1716905753432/

In [ ]:
# Load the input tensor
loaded_input_tensor = torch.load(d + 'input_tensor.pt')

# Load intermediate tensors
intermediate_tensors = {}
for layer in model.children():
    class_name = layer.__class__.__name__
    for module_idx in range(len(intermediate_outputs)):
        input_tensor = torch.load(d + f"{class_name}_{module_idx}_input.pt")
        output_tensor = torch.load(d + f"{class_name}_{module_idx}_output.pt")
        intermediate_tensors[f"{class_name}_{module_idx}"] = {
            "input": input_tensor,
            "output": output_tensor
        }

print("Loaded Input Tensor:", loaded_input_tensor)
for key, data in intermediate_tensors.items():
    print(f"Layer: {key}")
    print(f"Input: {data['input']}")
    print(f"Output: {data['output']}")
    print()


Loaded Input Tensor: tensor([[-1.0071,  0.7633],
        [ 1.3237,  0.0195],
        [-0.2534, -0.3866]])
Layer: Linear_0
Input: (tensor([[-1.0071,  0.7633],
        [ 1.3237,  0.0195],
        [-0.2534, -0.3866]]),)
Output: tensor([[ 0.4200, -0.4630,  0.6084,  0.7622],
        [ 0.4925, -0.7681, -1.0049,  0.3789],
        [ 0.1678, -0.1500, -0.4558,  0.0660]])

Layer: Linear_1
Input: (tensor([[0.4200, 0.0000, 0.6084, 0.7622],
        [0.4925, 0.0000, 0.0000, 0.3789],
        [0.1678, 0.0000, 0.0000, 0.0660]]),)
Output: tensor([[-0.2774],
        [-0.0251],
        [-0.0659]])

